In [1]:
import torch

#### DataLoader
https://pytorch.org/docs/1.1.0/data.html#torch.utils.data.DataLoader

##### torch.utils.data.DataLoader
  
Combines a dataset and a sampler,  
  
and provides **single- or multi-process iterators** over the dataset.  
  
(data size 만이 아니라)  
Batch size 와 iterating worker number 은 데이터를 불러올 때, 필요한 메모리 사이즈를 결정한다.  
*device에 대한 결정은 iterated 되어 나온 data에 직접 해야 한다.*
  
```  
torch.utils.data.DataLoader(
  dataset,  
  batch_size=1,  
  shuffle=False,  
  sampler=None,  
  batch_sampler=None,  
  num_workers=0,  
  collate_fn=<function default_collate>,  
  pin_memory=False,  
  drop_last=False,  
  timeout=0,  
  worker_init_fn=None)
```
parameters
* **dataset (Dataset)** – dataset from which to load the data.
  
  
* **batch_size** (int, optional) – how many samples per batch to load (default: 1).
* **shuffle** (bool, optional) – set to True to have the data reshuffled at every epoch (default: False).
  
  
* sampler (Sampler, optional) – defines the strategy to draw samples from the dataset. If specified, shuffle must be False.
* batch_sampler (Sampler, optional) – like sampler, but returns a batch of indices at a time. Mutually exclusive with batch_size, shuffle, sampler, and drop_last.
  
  
* **num_workers** (int, optional) – how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process. (default: 0)
  
  
* collate_fn (callable, optional) – merges a list of samples to form a **mini-batch**.
* pin_memory (bool, optional) – If True, the data loader will copy tensors into **CUDA pinned memory** before returning them. If your data elements are a custom type, or your collate_fn returns a batch that is a custom type see the example below.
  
  
* drop_last (bool, optional) – set to True to drop the **last incomplete batch**, if the dataset size is not divisible by the batch size. If False and the size of dataset is not divisible by the batch size, then the last batch will be smaller. (default: False)
  
  
* timeout (numeric, optional) – if positive, the timeout value for collecting a batch from workers. Should always be non-negative. (default: 0)
* worker_init_fn (callable, optional) – If not None, this will be called on each worker subprocess with the worker id (an int in [0, num_workers - 1]) as input, after seeding and before data loading. (default: None)

#### num_workers != 0
`DataLoader`를 위한 `iterator`가 얻어질 때마다 `worker process`가 만들어진다.  
각 `worker process`에 `collate_fn`과 `worker_init_fn`가 전달되어지고, batch list 를 만들 때 (`collate_fn`), 그리고 seeding 과 data loading 사이에 (`worker_init_fn`) 호출된다.  
  
shuffle randomization 은 main process에 일어나고, iteration이 끝나면 모든 workers가 shutdown 된다.
  
worker 는 python multiprocessing 에 의존하고, 동작은 운영체제 마다 다를 수 있다. (유닉스와 윈도우는 다르다.)
이 떄 *window compatible* 을 보증하기 위해 아래 2가지를 지켜야 한다.  
* main script's code 를 거의 전부 `if __name__ == '__main__':` 블럭 안에 넣는다. worker 에 대한 로직이 다시 실행되지 않게 하기 위해서라고 함.
* 단, `collate_fn`, `worker_init_fn` 또는 어떤 *custom dataset code* 를 top level def로, `__main__` 밖에서 정의해야 한다. 이렇게 해서 모든 worker 들이 접근할 수 있게 한다.
  
worker 각각이 각각의 random seed 를 갖는데, 다른 라이브러리에 사용되는 seed 중복될 수 있다. 이렇게 되어 각 시드가 같은 난수를 생성하게 될 때에 이를 피하고자 한다면, `worker_init_fn` 안에 `torch.initial_seed()`를 사용 할 수 있다.

또 `spawn`이 사용되는 경우, (윈도우 환경에서는 `fork()` 대신에  `spawn()` 에서 사용됨)  
lambda function 같은 unpickable object를 `worker_init_fn` 으로 사용할 수 없다.

In [2]:
from torch.utils.data import DataLoader
help(DataLoader)

Help on class DataLoader in module torch.utils.data.dataloader:

class DataLoader(builtins.object)
 |  DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=<function default_collate at 0x7f6b430c80d0>, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None)
 |  
 |  Data loader. Combines a dataset and a sampler, and provides
 |  single- or multi-process iterators over the dataset.
 |  
 |  Arguments:
 |      dataset (Dataset): dataset from which to load the data.
 |      batch_size (int, optional): how many samples per batch to load
 |          (default: ``1``).
 |      shuffle (bool, optional): set to ``True`` to have the data reshuffled
 |          at every epoch (default: ``False``).
 |      sampler (Sampler, optional): defines the strategy to draw samples from
 |          the dataset. If specified, ``shuffle`` must be False.
 |      batch_sampler (Sampler, optional): like sampler, but returns a batch of
 |          

1. using dataloader

In [3]:
'''
cats = CatDogDataset(cat_files, train_dir, transform = data_transform)
dogs = CatDogDataset(dog_files, train_dir, transform = data_transform)

catdogs = ConcatDataset([cats, dogs])
'''

# dataloader = DataLoader(catdogs, batch_size = 6, shuffle=True, num_workers=0)

'''
for epoch in range(epochs):
    for samples, labels in dataloader:
        samples, labels = samples.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(samples)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        scheduler.step()
'''

'\nfor epoch in range(epochs):\n    for samples, labels in dataloader:\n        samples, labels = samples.to(device), labels.to(device)\n        optimizer.zero_grad()\n        output = model(samples)\n        loss = criterion(output, labels)\n        loss.backward()\n        optimizer.step()\n        total_loss += loss.item()\n        scheduler.step()\n'

2. memory pinning

memory pinning 은 **cpu -> gpu data transfer** 시에 아주 유용하다.  
https://mkblog.co.kr/2017/03/07/nvidia-gpu-pinned-host-memory-cuda/

그런데 pytorch 에서는 `tensor` 에 대해서만 로직이 동작한다고 한다. 어떤 `batch`나 `custom data types` 을 위해서는 `custom type` 안에 `pin_memory()`를 정의해줘야 한다.

In [ ]:
class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)

    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self

def collate_wrapper(batch):
    return SimpleCustomBatch(batch)

inps = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
tgts = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
dataset = TensorDataset(inps, tgts)

loader = DataLoader(dataset, batch_size=2, collate_fn=collate_wrapper,
                    pin_memory=True)

for batch_ndx, sample in enumerate(loader):
    print(sample.inp.is_pinned())
    print(sample.tgt.is_pinned())
